In [ ]:
# @title Setup và Import thư viện
%pip install transformers
%pip install flask_ngrok
%pip install flask-cors
%pip install pyngrok

# Restart runtime nếu đây là lần đầu cài đặt các package
import os
if not os.path.exists('/root/.ngrok2/ngrok.yml'):
    print("Please restart runtime after installing packages!")

import torch
from transformers import MarianMTModel, MarianTokenizer
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from pyngrok import ngrok
import logging
import warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
# @title Setup Ngrok Authentication
# @markdown Nhập Ngrok Authtoken của bạn (đăng ký tại ngrok.com)
NGROK_AUTH_TOKEN = "" # @param {type:"string"}

if NGROK_AUTH_TOKEN:
    os.system(f'ngrok authtoken {NGROK_AUTH_TOKEN}')
else:
    print("Please provide your ngrok authtoken!")

In [ ]:
# @title Khởi tạo model và tokenizer
def load_model():
    try:
        logging.info("Loading model and tokenizer...")
        tokenizer = MarianTokenizer.from_pretrained("lmh2011/marianMT-finetuned-en-vi")
        model = MarianMTModel.from_pretrained("lmh2011/marianMT-finetuned-en-vi")
        logging.info("Model loaded successfully")
        return tokenizer, model
    except Exception as e:
        logging.error(f"Error loading model: {str(e)}")
        raise

tokenizer, model = load_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"Using device: {device}")
model = model.to(device)

In [ ]:
# @title Hàm dịch văn bản
def translate_text(text):
    try:
        # Tokenize
        inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)

        # Generate translation
        with torch.no_grad():
            translated = model.generate(**inputs, max_length=512)

        # Decode
        translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

        return translated_text
    except Exception as e:
        logging.error(f"Translation error: {str(e)}")
        raise

In [ ]:
# @title Khởi tạo Flask app
app = Flask(__name__)
CORS(app)

# Health check endpoint
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({'status': 'healthy', 'model': 'marianMT-finetuned-en-vi'})

@app.route('/translate', methods=['POST'])
def translate():
    try:
        data = request.get_json()

        if not data:
            return jsonify({'error': 'No JSON data provided'}), 400

        if 'text' not in data:
            return jsonify({'error': 'Missing text field'}), 400

        source_text = data['text']

        if not isinstance(source_text, str):
            return jsonify({'error': 'Text must be a string'}), 400

        if not source_text.strip():
            return jsonify({'error': 'Text cannot be empty'}), 400

        logging.info(f"Processing translation request: {source_text[:100]}...")
        translated_text = translate_text(source_text)
        logging.info("Translation completed successfully")

        return jsonify({
            'source_text': source_text,
            'translated_text': translated_text
        })

    except Exception as e:
        logging.error(f"Error processing request: {str(e)}")
        return jsonify({'error': str(e)}), 500

In [ ]:
# @title Chạy server
# Đóng tất cả tunnel hiện có
ngrok.kill()

# Tạo tunnel mới
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Chạy Flask app
if __name__ == '__main__':
    logging.info("Starting server...")
    app.run(port=5000)